## Mount Google Drive in Colab

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## Perform imports

In [0]:
import numpy as np
%tensorflow_version 1.x
import keras
from keras.layers import Input, Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.models import Model
from keras.optimizers import SGD
import json

TensorFlow 1.x selected.


Using TensorFlow backend.


## Load balanced dataset

In [0]:
data = np.load('/content/drive/My Drive/Final Capstone/Balanced Data.npy')

## Separate image data from label data

In [0]:
# image data
X = data[:, :-1]
# label data
Y = data[:, -1:]
del data

## Reshape image and label data and create one-hot encoding for label data

In [0]:
# reshape image data to 50 x 50 x 3 format
X = np.reshape(X, (X.shape[0], 50, 50, 3), order='C')
# reshape label data to 1-D array
Y = np.reshape(Y, (Y.shape[0]))
# create one-hot encoding for label data
Y = keras.utils.to_categorical(Y, 2, 'float32')

## Convolutional Neural Network models

Twelve different architectures for simple 2-D convolutional neural networks are created and fit to the training data based on a set of common hyperparameters.  After each epoch, the models are evaluated based on their accuracy in predicting the validation set.  These sets are the same as used in the random forest models.  Each architecture is run for a minimum of 5 epochs and a maximum of 20 epochs.  At the end of each epoch after the fourth epoch, the validation accuracy of the recently completed epoch is compared to the validation accuracy of the previous epoch.  If the validation accuracy decreases, the training of the model is terminated.  For each model, a dictionary recording the results of the epochs is saved as a .json file in the directory /content/drive/My Drive/Final Capstone/Model Logs/.

In [0]:
# fraction of data to use for validation--taken from the bottom of the dataset by default in Keras
val_split = 0.2
# number of filters used in the Conv2D layers
filters = 32
# kernel size used in the Conv2D layers
kernel_size = (3, 3)
# pool size used in MaxPooling2D layers
pool_size = (2, 2)
# dropout rate used in Dropout
dropout_rate = 0.25
# width of hidden layer used prior to prediction dense layer
hidden_dense_size = 64
# initial learning rate used in optimizer
initial_lr = 0.01

In [0]:
# function used to train models
def optimize_model(model, X, Y, val_split, initial_lr):
  # counter for # of epochs model has been trained on
  epoch_count = 0
  # minimum # of epochs
  min_epochs = 5
  # maximum # of epochs
  max_epochs = 20
  # initialize dictionary to save training results
  history_dict = None
  # instantiate stochastic gradient descent optimizer with initial learning rate
  sgd = SGD(lr=initial_lr)
  # save learning rate for use in later updates
  sgd_lr = sgd.get_config()['lr']
  # while loop based on model train accuracy convergence
  while (True):
    # while loop based on learning rate reduction
    while (True):
      # complile model
      model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
      # save model weights before training next epoch
      old_weights = model.get_weights()
      # train model for one epoch
      history = model.fit(X, Y, epochs=1, validation_split=val_split, verbose=1)

      # save results of epoch if it is the first epoch (i.e., nothing written to history_dict yet)
      if (history_dict == None):
        history_dict = history.history
        break
      # logic to determine whether to save epoch if it is not the first epoch
      else:
        # value of training loss from previous epoch
        last_loss = history_dict['loss'][len(history_dict['loss']) - 1]
        # value of training loss from epoch just trained
        curr_loss = history.history['loss'][0]
        # keep results of epoch just completed if training loss decreases or stays the same
        if (curr_loss <= last_loss):
          for key in history_dict.keys():
            history_dict[key].append(history.history[key][0])
          break
        # reduce learning rate if training loss increased, reset weights to previous, and rerun epoch
        else:
          sgd = SGD(lr=sgd_lr/2)
          sgd_lr = sgd.get_config()['lr']
          model.set_weights(old_weights)

    # increment epoch count after an epoch that reduces or maintains training loss is achieved
    epoch_count += 1

    # logic of whether to terminate training
    if (epoch_count >= min_epochs):
      # terminate if max numbers of epochs have been trained
      if (epoch_count == max_epochs):
        break
      else:
        # terminate if validation accuracy decreased (if min number of epochs have been trained)
        if (history_dict['val_acc'][epoch_count - 1] < history_dict['val_acc'][epoch_count - 2]):
          break
  # return dictionary containing training history
  return history_dict

In [0]:
# Conv2D, Flatten, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Flatten(data_format='channels_last')(output_layer1)
output_layer3 = Dense(units=2, activation='softmax')(output_layer2)
model = Model(input_layer, output_layer3)

output_dict = optimize_model(model, X, Y, val_split, tolerance, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/c_f_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# Conv2D, MaxPooling, Flatten, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer1)
output_layer3 = Flatten(data_format='channels_last')(output_layer2)
output_layer4 = Dense(units=2, activation='softmax')(output_layer3)
model = Model(input_layer, output_layer4)

output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/c_mp_f_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# Conv2D, MaxPooling, Flatten, Dropout, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer1)
output_layer3 = Flatten(data_format='channels_last')(output_layer2)
output_layer4 = Dropout(rate=dropout_rate)(output_layer3)
output_layer5 = Dense(units=2, activation='softmax')(output_layer4)
model = Model(input_layer, output_layer5)

output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/c_mp_f_dr_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# Conv2D, Flatten, Dense, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Flatten(data_format='channels_last')(output_layer1)
output_layer3 = Dense(units=hidden_dense_size, activation='relu')(output_layer2)
output_layer4 = Dense(units=2, activation='softmax')(output_layer3)
model = Model(input_layer, output_layer4)
output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/c_f_d_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# Conv2D, MaxPooling, Flatten, Dense, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer1)
output_layer3 = Flatten(data_format='channels_last')(output_layer2)
output_layer4 = Dense(units=hidden_dense_size, activation='relu')(output_layer3)
output_layer5 = Dense(units=2, activation='softmax')(output_layer4)
model = Model(input_layer, output_layer5)

output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/c_mp_f_d_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# Conv2D, MaxPooling, Flatten, Dropout, Dense, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer1)
output_layer3 = Flatten(data_format='channels_last')(output_layer2)
output_layer4 = Dropout(rate=dropout_rate)(output_layer3)
output_layer5 = Dense(units=hidden_dense_size, activation='relu')(output_layer4)
output_layer6 = Dense(units=2, activation='softmax')(output_layer5)
model = Model(input_layer, output_layer6)

output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/c_mp_f_dr_d_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# Conv2D, Conv2D, Flatten, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(output_layer1)
output_layer3 = Flatten(data_format='channels_last')(output_layer2)
output_layer4 = Dense(units=2, activation='softmax')(output_layer3)
model = Model(input_layer, output_layer4)

output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/c_c_f_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# Conv2D, Conv2D, MaxPooling, Flatten, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(output_layer1)
output_layer3 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer2)
output_layer4 = Flatten(data_format='channels_last')(output_layer3)
output_layer5 = Dense(units=2, activation='softmax')(output_layer4)
model = Model(input_layer, output_layer5)

output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/c_c_mp_f_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# Conv2D, Conv2D, MaxPooling, Flatten, Dropout, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(output_layer1)
output_layer3 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer2)
output_layer4 = Flatten(data_format='channels_last')(output_layer3)
output_layer5 = Dropout(rate=dropout_rate)(output_layer4)
output_layer6 = Dense(units=2, activation='softmax')(output_layer5)
model = Model(input_layer, output_layer6)

output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/c_c_mp_f_dr_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# Conv2D, Conv2D, Flatten, Dense, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(output_layer1)
output_layer3 = Flatten(data_format='channels_last')(output_layer2)
output_layer4 = Dense(units=hidden_dense_size, activation='relu')(output_layer3)
output_layer5 = Dense(units=2, activation='softmax')(output_layer4)
model = Model(input_layer, output_layer5)

output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/c_c_f_d_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# Conv2D, Conv2D, MaxPooling, Flatten, Dense, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(output_layer1)
output_layer3 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer2)
output_layer4 = Flatten(data_format='channels_last')(output_layer3)
output_layer5 = Dense(units=hidden_dense_size, activation='relu')(output_layer4)
output_layer6 = Dense(units=2, activation='softmax')(output_layer5)
model = Model(input_layer, output_layer6)

output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/c_c_mp_f_d_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# Conv2D, Conv2D, MaxPooling, Flatten, Dropout, Dense, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(output_layer1)
output_layer3 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer2)
output_layer4 = Flatten(data_format='channels_last')(output_layer3)
output_layer5 = Dropout(rate=dropout_rate)(output_layer4)
output_layer6 = Dense(units=hidden_dense_size, activation='relu')(output_layer5)
output_layer7 = Dense(units=2, activation='softmax')(output_layer6)
model = Model(input_layer, output_layer7)

output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/c_c_mp_f_dr_d_d.json', 'w') as f:
  json.dump(output_dict, f)

In addition to simple 2-D Convolutional Neural Network architectures, transfer learning using VGG16 is explored.